<a href="https://colab.research.google.com/github/marika-rago/Progetto-ML/blob/main/realizzazione_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import os
import librosa
import soundfile as sf
import numpy as np
from pydub import AudioSegment
import matplotlib.pyplot as plt
import random

In [16]:
input_path = "/content/drive/MyDrive/Progetto ML/mus18db/train"
hq_dir = "/content/drive/MyDrive/Progetto ML/mus18db_ridotto/HQ"

os.makedirs(hq_dir, exist_ok=True)


In [17]:
# numero di brani da selezionare
n_files = 10   # cambia questo numero a piacere

# settiamo un seed per la riproducibilità
random.seed(42)

# lista di tutti i file .wav
all_files = [f for f in os.listdir(input_path) if os.path.isdir(os.path.join(input_path, f))]

# selezione random di n_files (senza ripetizioni)
files = random.sample(all_files, n_files)

print(f"Brani selezionati ({n_files}):")
for f in files:
    print("-", f)

Brani selezionati (10):
- Chris Durban - Celebrate
- The Scarlet Brand - Les Fleurs Du Mal
- Young Griffo - Blood To Bone
- Alexander Ross - Goodbye Bolero
- Music Delta - Reggae
- North To Alaska - All The Same
- Phre The Eon - Everybody's Falling Apart
- Strand Of Oaks - Spacestation
- The Long Wait - Back Home To Blue
- AvaLuna - Waterduct


In [18]:
clip_duration = 10  # secondi
n_clips_per_track = 3  # quante clip estrarre per ogni brano

for f in files:
    file_path = os.path.join(input_path, f, "mixture.wav") # percorso completo file audio
    # "mixture.wav" è la canzone completa

    if not os.path.exists(file_path):
        continue

    y, sr = librosa.load(file_path, sr=None, mono=False)
    # y è il waveform (array NumPy) normalizzato in [-1, 1]
    # sr è il sample rate originale del file

    total_duration = librosa.get_duration(y=y, sr=sr)

    for i in range(n_clips_per_track):
        # scegle inizio casuale evitando di uscire dalla fine
        max_start = total_duration - clip_duration
        if max_start <= 0:
            # se il brano è più corto di 10s, salta
            continue

        # Converte l’istante di inizio in intero e calcola la fine
        start_time = random.uniform(0, max_start)
        start = int(start_time * sr)
        end = min(start + clip_duration * sr, y.shape[1])
        clip = y[:, start:end]  # stereo: (2, n_samples)

        out_file = os.path.join(hq_dir, f"{f.replace(' ', '_')}_clip{i}.wav")

        # soundfile vuole (n_samples, n_channels)
        sf.write(out_file, clip.T, sr)
